In [1]:
import sys
import subprocess

def install_if_missing(package):
    try:
        __import__(package)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

for package in ['ipywidgets', 'matplotlib']:
    install_if_missing(package)

print("Packages ready")


Packages ready


In [2]:
import gymnasium as gym
from stable_baselines3 import SAC, TD3, A2C
import ipywidgets as widgets
from IPython.display import display, clear_output
import time
import os
import subprocess
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported")


Libraries imported


In [3]:
import mujoco
from gymnasium.envs.mujoco.mujoco_rendering import WindowViewer

original_create_overlay = WindowViewer._create_overlay

def patched_create_overlay(self):
    if hasattr(self.data, 'solver_niter') and not hasattr(self.data, 'solver_iter'):
        self.data.solver_iter = self.data.solver_niter
    return original_create_overlay(self)

WindowViewer._create_overlay = patched_create_overlay
print("Compatibility fix applied")


Compatibility fix applied


In [ ]:
class MujocoViewer:
    def __init__(self):
        self.process = None
        self.create_widgets()
        self.create_layout()
        
    def get_models_by_algorithm(self):
        models_dir = 'models'
        algorithm_models = {'A2C': [], 'SAC': [], 'TD3': []}
        
        if os.path.exists(models_dir):
            for file in os.listdir(models_dir):
                if file.endswith('.zip'):
                    model_name = file.replace('.zip', '')
                    if 'A2C' in model_name.upper():
                        algorithm_models['A2C'].append(model_name)
                    elif 'SAC' in model_name.upper():
                        algorithm_models['SAC'].append(model_name)
                    elif 'TD3' in model_name.upper():
                        algorithm_models['TD3'].append(model_name)
        
        return algorithm_models
        
    def create_widgets(self):
        self.title = widgets.HTML(value='<h2>MuJoCo Model Viewer</h2>')
        
        self.algo_dropdown = widgets.Dropdown(
            options=['A2C', 'SAC', 'TD3'], value='A2C',
            description='Algorithm:', style={'description_width': '80px'}
        )
        
        self.model_dropdown = widgets.Dropdown(
            options=['No models found'],
            description='Model:', style={'description_width': '80px'}
        )
        
        self.env_dropdown = widgets.Dropdown(
            options=['Humanoid-v4', 'Humanoid-v5'], value='Humanoid-v4',
            description='Environment:', style={'description_width': '80px'}
        )
        
        self.episodes_slider = widgets.IntSlider(
            value=1, min=1, max=10,
            description='Episodes:', style={'description_width': '80px'}
        )
        
        self.steps_slider = widgets.IntSlider(
            value=1000, min=100, max=5000, step=100,
            description='Max Steps:', style={'description_width': '80px'}
        )
        
        self.record_video = widgets.Checkbox(value=False, description='Record Video')
        
        self.execute_btn = widgets.Button(
            description='Run Viewer', button_style='success', layout={'width': '120px'}
        )
        
        self.stop_btn = widgets.Button(
            description='Stop', button_style='danger', layout={'width': '80px'}
        )
        
        self.summary_btn = widgets.Button(
            description='Summary', button_style='info', layout={'width': '80px'}
        )
        
        self.visualize_btn = widgets.Button(
            description='Visualize', button_style='warning', layout={'width': '80px'}
        )
        
        self.refresh_btn = widgets.Button(
            description='Refresh', button_style='primary', layout={'width': '80px'}
        )
        
        self.status_label = widgets.HTML(value='Ready')
        self.output = widgets.Output(layout={'height': '300px', 'overflow': 'scroll'})
        
        self.algo_dropdown.observe(self.update_models, names='value')
        self.execute_btn.on_click(self.execute_command)
        self.stop_btn.on_click(self.stop_execution)
        self.summary_btn.on_click(self.show_summary)
        self.visualize_btn.on_click(self.show_visualizations)
        self.refresh_btn.on_click(self.refresh_models)
        
        self.update_models({'new': self.algo_dropdown.value})
        
    def update_models(self, change):
        algorithm = change['new']
        models_by_algo = self.get_models_by_algorithm()
        
        if models_by_algo[algorithm]:
            def extract_number(model_name):
                import re
                numbers = re.findall(r'\d+', model_name)
                return int(numbers[-1]) if numbers else 0
            
            sorted_models = sorted(models_by_algo[algorithm], key=extract_number, reverse=True)
            self.model_dropdown.options = sorted_models
            self.model_dropdown.value = sorted_models[0]
        else:
            self.model_dropdown.options = [f'No {algorithm} models found']
            
    def create_layout(self):
        params_box = widgets.VBox([
            widgets.HBox([self.algo_dropdown, self.model_dropdown, self.env_dropdown]),
            widgets.HBox([self.episodes_slider, self.steps_slider, self.record_video])
        ])
        
        controls_box = widgets.HBox([self.execute_btn, self.stop_btn])
        analysis_box = widgets.HBox([self.summary_btn, self.visualize_btn, self.refresh_btn])
        
        self.main_layout = widgets.VBox([
            self.title, params_box, controls_box, analysis_box, self.status_label, self.output
        ])
        
    def execute_command(self, btn):
        with self.output:
            clear_output()
            
            model_name = self.model_dropdown.value
            algorithm = self.algo_dropdown.value
            environment = self.env_dropdown.value
            episodes = self.episodes_slider.value
            max_steps = self.steps_slider.value
            record_video = self.record_video.value
            
            if 'No' in model_name:
                print(f"No {algorithm} models available")
                return
                
            model_path = f"models/{model_name}.zip"
            
            print(f"Model: {model_name} | Algorithm: {algorithm} | Environment: {environment}")
            print(f"Episodes: {episodes} | Max Steps: {max_steps} | Record: {record_video}")
            print("="*50)
            
            self.status_label.value = 'Running...'
            
            try:
                python_script = f"""
import gymnasium as gym
from stable_baselines3 import SAC, TD3, A2C

env_name = '{environment}'
algo = '{algorithm}'
model_path = '{model_path}'
episodes = {episodes}
max_steps = {max_steps}
record_video = {record_video}

print(f"Loading {{algo}} model...")

if record_video:
    from gymnasium.wrappers import RecordVideo
    env = gym.make(env_name, render_mode='rgb_array')
    env = RecordVideo(env, video_folder='videos', episode_trigger=lambda x: True)
else:
    env = gym.make(env_name, render_mode='human')

if algo == 'SAC':
    model = SAC.load(model_path, env=env)
elif algo == 'TD3':
    model = TD3.load(model_path, env=env)
elif algo == 'A2C':
    model = A2C.load(model_path, env=env)

for episode in range(episodes):
    obs, _ = env.reset()
    total_reward = 0
    
    for step in range(max_steps):
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, truncated, _ = env.step(action)
        total_reward += reward
        
        if done or truncated:
            break
    
    print(f"Episode {{episode + 1}}: {{total_reward:.2f}} reward, {{step + 1}} steps")

env.close()
print("Execution completed")
"""
                
                self.process = subprocess.Popen(
                    ['python', '-c', python_script],
                    stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
                    text=True, bufsize=1, universal_newlines=True
                )
                
                for line in iter(self.process.stdout.readline, ''):
                    if line:
                        print(line.strip())
                
                self.process.wait()
                self.status_label.value = 'Completed' if self.process.returncode == 0 else 'Failed'
                    
            except Exception as e:
                print(f"Error: {str(e)}")
                self.status_label.value = f'Error: {str(e)}'
                
    def stop_execution(self, btn):
        if self.process and self.process.poll() is None:
            self.process.terminate()
            self.status_label.value = 'Stopped'
            print("Execution stopped")
    
    def refresh_models(self, btn):
        """Refresh the models list"""
        with self.output:
            clear_output()
            print("Refreshing models...")
            self.update_models({'new': self.algo_dropdown.value})
            print("Models refreshed")
            
    def show_summary(self, btn):
        """Show summary of all available models"""
        with self.output:
            clear_output()
            try:
                print("MODEL SUMMARY")
                print("="*50)
                
                models_by_algo = self.get_models_by_algorithm()
                total_models = 0
                
                for algo, models in models_by_algo.items():
                    print(f"\n{algo} Models ({len(models)}):")
                    if models:
                        for model in sorted(models, reverse=True):
                            model_path = f"models/{model}.zip"
                            if os.path.exists(model_path):
                                size = os.path.getsize(model_path) / (1024 * 1024)
                                print(f"  • {model} ({size:.1f} MB)")
                                total_models += 1
                    else:
                        print(f"  No {algo} models found")
                
                print(f"\nTotal Models: {total_models}")
                print(f"Models Directory: {os.path.abspath('models')}")
                
                if os.path.exists('videos'):
                    video_count = len([f for f in os.listdir('videos') if f.endswith('.mp4')])
                    print(f"Available Videos: {video_count}")
                    
            except Exception as e:
                print(f"Error generating summary: {str(e)}")
    
    def show_visualizations(self, btn):
        """Show performance visualizations"""
        with self.output:
            clear_output()
            try:
                import matplotlib.pyplot as plt
                import numpy as np
                
                print("PERFORMANCE VISUALIZATIONS")
                print("="*50)
                
                models_by_algo = self.get_models_by_algorithm()
                
                # Create sample performance data for visualization
                fig, axes = plt.subplots(2, 2, figsize=(12, 8))
                
                # Model count by algorithm
                algos = list(models_by_algo.keys())
                counts = [len(models) for models in models_by_algo.values()]
                
                axes[0,0].bar(algos, counts, color=['#1f77b4', '#ff7f0e', '#2ca02c'])
                axes[0,0].set_title('Models by Algorithm')
                axes[0,0].set_ylabel('Number of Models')
                
                # Model file sizes
                all_models = []
                all_sizes = []
                all_algos = []
                
                for algo, models in models_by_algo.items():
                    for model in models[:3]:  # Show top 3 models per algorithm
                        model_path = f"models/{model}.zip"
                        if os.path.exists(model_path):
                            size = os.path.getsize(model_path) / (1024 * 1024)
                            all_models.append(model[:15] + '...' if len(model) > 15 else model)
                            all_sizes.append(size)
                            all_algos.append(algo)
                
                if all_models:
                    colors = {'A2C': '#1f77b4', 'SAC': '#ff7f0e', 'TD3': '#2ca02c'}
                    bar_colors = [colors.get(algo, '#gray') for algo in all_algos]
                    
                    axes[0,1].barh(range(len(all_models)), all_sizes, color=bar_colors)
                    axes[0,1].set_yticks(range(len(all_models)))
                    axes[0,1].set_yticklabels(all_models)
                    axes[0,1].set_title('Model File Sizes (MB)')
                    axes[0,1].set_xlabel('Size (MB)')
                
                # Training progress simulation
                steps = np.arange(0, 1000000, 25000)
                sac_rewards = 1000 + 500 * np.log(steps + 1) + np.random.normal(0, 50, len(steps))
                a2c_rewards = 800 + 300 * np.log(steps + 1) + np.random.normal(0, 40, len(steps))
                td3_rewards = 1200 + 400 * np.log(steps + 1) + np.random.normal(0, 60, len(steps))
                
                axes[1,0].plot(steps, sac_rewards, label='SAC', color='#ff7f0e')
                axes[1,0].plot(steps, a2c_rewards, label='A2C', color='#1f77b4')
                axes[1,0].plot(steps, td3_rewards, label='TD3', color='#2ca02c')
                axes[1,0].set_title('Training Progress (Simulated)')
                axes[1,0].set_xlabel('Training Steps')
                axes[1,0].set_ylabel('Average Reward')
                axes[1,0].legend()
                axes[1,0].grid(True, alpha=0.3)
                
                # Algorithm comparison
                metrics = ['Sample Efficiency', 'Stability', 'Performance', 'Speed']
                sac_scores = [8, 7, 9, 6]
                a2c_scores = [5, 8, 6, 9]
                td3_scores = [7, 6, 8, 7]
                
                x = np.arange(len(metrics))
                width = 0.25
                
                axes[1,1].bar(x - width, sac_scores, width, label='SAC', color='#ff7f0e')
                axes[1,1].bar(x, a2c_scores, width, label='A2C', color='#1f77b4')
                axes[1,1].bar(x + width, td3_scores, width, label='TD3', color='#2ca02c')
                
                axes[1,1].set_title('Algorithm Comparison')
                axes[1,1].set_ylabel('Score (1-10)')
                axes[1,1].set_xticks(x)
                axes[1,1].set_xticklabels(metrics, rotation=45)
                axes[1,1].legend()
                axes[1,1].set_ylim(0, 10)
                
                plt.tight_layout()
                plt.show()
                
                print("\nVisualization complete!")
                
            except Exception as e:
                print(f"Error generating visualizations: {str(e)}")

viewer = MujocoViewer()
display(viewer.main_layout)


In [5]:
from anthropometric_analysis import AnthropometricAnalyzer, extract_trajectory_data_from_env
import matplotlib.pyplot as plt

class AnthropometricAnalysisUI:
    def __init__(self):
        self.create_widgets()
        self.create_layout()
        
    def get_models_by_algorithm(self):
        models_dir = 'models'
        algorithm_models = {'A2C': [], 'SAC': [], 'TD3': []}
        
        if os.path.exists(models_dir):
            for file in os.listdir(models_dir):
                if file.endswith('.zip'):
                    model_name = file.replace('.zip', '')
                    if 'A2C' in model_name.upper():
                        algorithm_models['A2C'].append(model_name)
                    elif 'SAC' in model_name.upper():
                        algorithm_models['SAC'].append(model_name)
                    elif 'TD3' in model_name.upper():
                        algorithm_models['TD3'].append(model_name)
        
        return algorithm_models
        
    def create_widgets(self):
        self.title = widgets.HTML(value='<h3>Anthropometric Analysis</h3>')
        
        self.algo_dropdown = widgets.Dropdown(
            options=['A2C', 'SAC', 'TD3'], value='A2C',
            description='Algorithm:', style={'description_width': '80px'}
        )
        
        self.model_dropdown = widgets.Dropdown(
            options=['No models found'],
            description='Model:', style={'description_width': '80px'}
        )
        
        self.env_dropdown = widgets.Dropdown(
            options=['Humanoid-v4', 'Humanoid-v5'], value='Humanoid-v4',
            description='Environment:', style={'description_width': '80px'}
        )
        
        self.steps_slider = widgets.IntSlider(
            value=1000, min=100, max=5000, step=100,
            description='Steps:', style={'description_width': '80px'}
        )
        
        self.analysis_type = widgets.SelectMultiple(
            options=['Joint Angles', 'Gait Parameters', 'Reach Kinematics', 'Movement Smoothness'],
            value=['Joint Angles', 'Gait Parameters'],
            description='Analysis:', style={'description_width': '80px'},
            layout={'height': '80px'}
        )
        
        self.analyze_btn = widgets.Button(
            description='Run Analysis', button_style='success', layout={'width': '120px'}
        )
        self.clear_btn = widgets.Button(
            description='Clear', button_style='warning', layout={'width': '80px'}
        )
        
        self.status_label = widgets.HTML(value='Ready')
        self.output = widgets.Output(layout={'height': '300px', 'overflow': 'scroll'})
        
        self.analyze_btn.on_click(self.run_analysis)
        self.clear_btn.on_click(self.clear_output)
        self.algo_dropdown.observe(self.update_models, names='value')
        
        self.update_models({'new': self.algo_dropdown.value})
        
    def update_models(self, change):
        algorithm = change['new']
        models_by_algo = self.get_models_by_algorithm()
        
        if models_by_algo[algorithm]:
            sorted_models = sorted(models_by_algo[algorithm], reverse=True)
            self.model_dropdown.options = sorted_models
            self.model_dropdown.value = sorted_models[0]
        else:
            self.model_dropdown.options = [f'No {algorithm} models found']
    
    def create_layout(self):
        params_box = widgets.VBox([
            widgets.HBox([self.algo_dropdown, self.model_dropdown, self.env_dropdown]),
            widgets.HBox([self.steps_slider, self.analysis_type])
        ])
        controls_box = widgets.HBox([self.analyze_btn, self.clear_btn])
        
        self.main_layout = widgets.VBox([
            self.title, params_box, controls_box, self.status_label, self.output
        ])
    
    def run_analysis(self, btn):
        with self.output:
            clear_output()
            try:
                self.status_label.value = 'Running Analysis...'
                
                model_name = self.model_dropdown.value
                algorithm = self.algo_dropdown.value
                environment = self.env_dropdown.value
                steps = self.steps_slider.value
                analysis_types = list(self.analysis_type.value)
                
                if 'No' in model_name:
                    print(f"No {algorithm} models available")
                    return
                
                print(f"Model: {model_name} | Algorithm: {algorithm} | Environment: {environment} | Steps: {steps}")
                
                import gymnasium as gym
                from stable_baselines3 import SAC, TD3, A2C
                
                model_path = f"models/{model_name}.zip"
                env = gym.make(environment, render_mode=None)
                
                if algorithm == 'SAC':
                    model = SAC.load(model_path, env=env)
                elif algorithm == 'TD3':
                    model = TD3.load(model_path, env=env)
                elif algorithm == 'A2C':
                    model = A2C.load(model_path, env=env)
                
                trajectory_data = extract_trajectory_data_from_env(env, model, steps)
                analyzer = AnthropometricAnalyzer()
                
                if 'Joint Angles' in analysis_types:
                    analyzer.analyze_joint_angles(trajectory_data.get('joint_angles', {}))
                if 'Gait Parameters' in analysis_types:
                    analyzer.analyze_gait_parameters(trajectory_data)
                if 'Reach Kinematics' in analysis_types:
                    analyzer.analyze_reach_kinematics(trajectory_data)
                if 'Movement Smoothness' in analysis_types:
                    analyzer.analyze_movement_smoothness(trajectory_data)
                
                results = analyzer.analyze_anthropometric_compliance(trajectory_data)
                analyzer.print_analysis_report(results)
                analyzer.generate_visualization(results)
                plt.show()
                
                env.close()
                self.status_label.value = 'Analysis Complete'
                
            except Exception as e:
                print(f"Error: {str(e)}")
                self.status_label.value = f'Error: {str(e)}'
    
    def clear_output(self, btn):
        with self.output:
            clear_output()
        self.status_label.value = 'Ready'

anthropometric_ui = AnthropometricAnalysisUI()
display(anthropometric_ui.main_layout)


In [6]:
from csv_input_handler import CSVInputHandler, load_csv_data, run_anthropometric_analysis_with_csv
import pandas as pd

class CSVInputUI:
    def __init__(self):
        self.csv_handler = CSVInputHandler()
        self.loaded_data = {}
        self.create_widgets()
        self.create_layout()
        
    def get_models_by_algorithm(self):
        models_dir = 'models'
        algorithm_models = {'A2C': [], 'SAC': [], 'TD3': []}
        
        if os.path.exists(models_dir):
            for file in os.listdir(models_dir):
                if file.endswith('.zip'):
                    model_name = file.replace('.zip', '')
                    if 'A2C' in model_name.upper():
                        algorithm_models['A2C'].append(model_name)
                    elif 'SAC' in model_name.upper():
                        algorithm_models['SAC'].append(model_name)
                    elif 'TD3' in model_name.upper():
                        algorithm_models['TD3'].append(model_name)
        
        return algorithm_models
        
    def create_widgets(self):
        self.title = widgets.HTML(value='<h3>CSV Input Handler</h3>')
        
        self.csv_type_dropdown = widgets.Dropdown(
            options=['anthropometric_params', 'trajectory_data', 'model_config', 'joint_angles', 'gait_data'],
            value='anthropometric_params', description='Type:', style={'description_width': '80px'}
        )
        self.file_path_text = widgets.Text(
            value='sample_csvs/anthropometric_params_sample.csv',
            description='File:', style={'description_width': '80px'}, layout={'width': '300px'}
        )
        
        self.validate_btn = widgets.Button(description='Validate', button_style='warning', layout={'width': '80px'})
        self.load_btn = widgets.Button(description='Load', button_style='success', layout={'width': '80px'})
        self.browse_btn = widgets.Button(description='Browse', button_style='info', layout={'width': '80px'})
        
        self.analysis_algo_dropdown = widgets.Dropdown(
            options=['A2C', 'SAC', 'TD3'], value='A2C', description='Algorithm:', style={'description_width': '80px'}
        )
        self.analysis_model_dropdown = widgets.Dropdown(
            options=['No models found'], description='Model:', style={'description_width': '80px'}
        )
        self.analysis_env_dropdown = widgets.Dropdown(
            options=['Humanoid-v4', 'Humanoid-v5'], value='Humanoid-v4', description='Environment:', style={'description_width': '80px'}
        )
        self.analysis_steps = widgets.IntSlider(
            value=1000, min=100, max=5000, step=100, description='Steps:', style={'description_width': '80px'}
        )
        
        self.run_enhanced_btn = widgets.Button(description='Enhanced Analysis', button_style='primary', layout={'width': '150px'})
        self.clear_btn = widgets.Button(description='Clear', button_style='danger', layout={'width': '80px'})
        
        self.data_display = widgets.HTML(value='No CSV data loaded')
        self.status_label = widgets.HTML(value='Ready')
        self.output = widgets.Output(layout={'height': '300px', 'overflow': 'scroll'})
        
        self.validate_btn.on_click(self.validate_csv)
        self.load_btn.on_click(self.load_csv)
        self.browse_btn.on_click(self.browse_samples)
        self.run_enhanced_btn.on_click(self.run_enhanced_analysis)
        self.clear_btn.on_click(self.clear_data)
        self.csv_type_dropdown.observe(self.update_sample_path, names='value')
        self.analysis_algo_dropdown.observe(self.update_models, names='value')
        
        self.update_models({'new': self.analysis_algo_dropdown.value})
        
    def update_models(self, change):
        algorithm = change['new']
        models_by_algo = self.get_models_by_algorithm()
        
        if models_by_algo[algorithm]:
            sorted_models = sorted(models_by_algo[algorithm], reverse=True)
            self.analysis_model_dropdown.options = sorted_models
            self.analysis_model_dropdown.value = sorted_models[0]
        else:
            self.analysis_model_dropdown.options = [f'No {algorithm} models found']
    
    def update_sample_path(self, change):
        csv_type = change['new']
        sample_path = f"sample_csvs/{csv_type}_sample.csv"
        if os.path.exists(sample_path):
            self.file_path_text.value = sample_path
    
    def create_layout(self):
        upload_section = widgets.VBox([
            widgets.HBox([self.csv_type_dropdown, self.file_path_text]),
            widgets.HBox([self.validate_btn, self.load_btn, self.browse_btn])
        ])
        analysis_section = widgets.VBox([
            widgets.HBox([self.analysis_algo_dropdown, self.analysis_model_dropdown, self.analysis_env_dropdown]),
            widgets.HBox([self.analysis_steps])
        ])
        controls_box = widgets.HBox([self.run_enhanced_btn, self.clear_btn])
        
        self.main_layout = widgets.VBox([
            self.title, upload_section, self.data_display, analysis_section,
            controls_box, self.status_label, self.output
        ])

    def validate_csv(self, btn):
        with self.output:
            clear_output()
            try:
                file_path = self.file_path_text.value
                csv_type = self.csv_type_dropdown.value
                
                if not os.path.exists(file_path):
                    print(f"File not found: {file_path}")
                    return
                
                is_valid = self.csv_handler.validate_csv_format(file_path, csv_type)
                if is_valid:
                    df = pd.read_csv(file_path)
                    print(f"Valid - Rows: {len(df)}, Columns: {', '.join(df.columns)}")
                    print(df.head())
                    self.status_label.value = 'Valid'
                else:
                    self.status_label.value = 'Invalid format'
            except Exception as e:
                print(f"Error: {str(e)}")
                self.status_label.value = f'Error: {str(e)}'
    
    def load_csv(self, btn):
        with self.output:
            clear_output()
            try:
                file_path = self.file_path_text.value
                csv_type = self.csv_type_dropdown.value
                data = load_csv_data(file_path, csv_type)
                
                if data:
                    self.loaded_data[csv_type] = data
                    print(f"Loaded {csv_type}")
                    self.update_data_display()
                    self.status_label.value = 'Loaded'
                else:
                    print("Load failed")
                    self.status_label.value = 'Failed'
            except Exception as e:
                print(f"Error: {str(e)}")
                self.status_label.value = f'Error: {str(e)}'
    
    def browse_samples(self, btn):
        with self.output:
            clear_output()
            print("Sample CSV Files:")
            sample_dir = 'sample_csvs'
            if os.path.exists(sample_dir):
                for file in os.listdir(sample_dir):
                    if file.endswith('.csv'):
                        print(f"{file}")
            else:
                print("Sample directory not found")
    
    def update_data_display(self):
        if self.loaded_data:
            content = 'Loaded: ' + ', '.join(self.loaded_data.keys())
            self.data_display.value = content
        else:
            self.data_display.value = 'No CSV data loaded'
    
    def run_enhanced_analysis(self, btn):
        with self.output:
            clear_output()
            try:
                if not self.loaded_data:
                    print("No CSV data loaded")
                    return
                
                model_name = self.analysis_model_dropdown.value
                algorithm = self.analysis_algo_dropdown.value
                environment = self.analysis_env_dropdown.value
                steps = self.analysis_steps.value
                
                if 'No' in model_name:
                    print(f"No {algorithm} models available")
                    return
                
                print(f"Running enhanced analysis: {model_name}")
                
                csv_inputs = {csv_type: f"sample_csvs/{csv_type}_sample.csv" for csv_type in self.loaded_data.keys()}
                
                results = run_anthropometric_analysis_with_csv(
                    model_name=model_name, algorithm=algorithm, environment=environment,
                    csv_inputs=csv_inputs, steps=steps
                )
                self.status_label.value = 'Complete'
                
            except Exception as e:
                print(f"Error: {str(e)}")
                self.status_label.value = f'Error: {str(e)}'
    
    def clear_data(self, btn):
        self.loaded_data = {}
        self.update_data_display()
        with self.output:
            clear_output()
        self.status_label.value = 'Ready'

csv_ui = CSVInputUI()
display(csv_ui.main_layout)
